A1. Write your own module for detecting the feature/attributefor the root note of a Decision Tree. Use Information gain as the impurity measure for identifying the root node. Assume that the features are categoricalor could be converted to categorical by binning. 

In [1]:
import numpy as np

def entropy(y):
    """Calculate the entropy of a label array y."""
    class_labels, counts = np.unique(y, return_counts=True)
    probabilities = counts / counts.sum()
    return -np.sum(probabilities * np.log2(probabilities))

def information_gain(X, y, feature_index):
    """Calculate the information gain of splitting on a given feature."""
    # Total entropy before split
    total_entropy = entropy(y)
    
    # Values and counts of the feature to split on
    values, counts = np.unique(X[:, feature_index], return_counts=True)
    weighted_entropy = 0
    
    for value, count in zip(values, counts):
        subset = y[X[:, feature_index] == value]
        weighted_entropy += (count / y.size) * entropy(subset)
    
    # Information gain is total entropy minus the weighted entropy of the split
    return total_entropy - weighted_entropy

def find_best_feature(X, y):
    """Identify the best feature by information gain."""
    gains = [information_gain(X, y, feature) for feature in range(X.shape[1])]
    return np.argmax(gains)

# Example usage:
# X = [[feature1, feature2, ...], [feature1, feature2, ...], ...]
# y = [class1, class2, class1, class2, ...]
# best_feature = find_best_feature(X, y)
# print("Best feature index:", best_feature)


A2. If the feature is continuous valued for A1, use equal widthor frequencybinning for converting the attribute to categorical valued.The binning type should be aparameter to the function built for binning. Write your own function for the binning task.The number of bins to be created should also be passed as aparameter to the function. Use function overloadingtoallow for usage of default parameters if no parameters are passed

In [2]:
def bin_continuous_feature(values, num_bins=3, method='width'):
    """Bin a continuous feature into categorical."""
    if method == 'width':
        bins = np.linspace(np.min(values), np.max(values), num=num_bins+1)
    elif method == 'frequency':
        bins = np.percentile(values, np.linspace(0, 100, num=num_bins+1))
    
    categorical = np.digitize(values, bins) - 1  # Convert to zero-indexed categories
    return categorical

def categorical_binning(X, feature_index, num_bins=3, method='width'):
    """Apply binning to a specified feature in the dataset X."""
    X_binned = X.copy()
    X_binned[:, feature_index] = bin_continuous_feature(X[:, feature_index], num_bins, method)
    return X_binned

# Example usage for a specific feature index and method
# X_binned = categorical_binning(X, feature_index=0, num_bins=4, method='frequency')


A3. Expand the above functions to built your own Decision Tree module.

In [3]:
class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

def build_decision_tree(X, y, depth=0, max_depth=3):
    if len(np.unique(y)) == 1 or depth == max_depth:
        # If all targets are the same or max depth reached, return a leaf node
        return DecisionTreeNode(value=np.bincount(y).argmax())
    
    best_feature = find_best_feature(X, y)
    thresholds = np.unique(X[:, best_feature])
    best_threshold = None
    best_ig = -1
    
    for threshold in thresholds:
        left_y = y[X[:, best_feature] <= threshold]
        right_y = y[X[:, best_feature] > threshold]
        ig = information_gain(X, y, best_feature)
        if ig > best_ig:
            best_ig = ig
            best_threshold = threshold
    
    if best_ig == 0:
        return DecisionTreeNode(value=np.bincount(y).argmax())

    left_subtree = build_decision_tree(X[X[:, best_feature] <= best_threshold], y[X[:, best_feature] <= best_threshold], depth+1, max_depth)
    right_subtree = build_decision_tree(X[X[:, best_feature] > best_threshold], y[X[:, best_feature] > best_threshold], depth+1, max_depth)
    
    return DecisionTreeNode(feature=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)

# Example of building a tree
# tree = build_decision_tree(X, y, max_depth=5)
